In [2]:
# main.py 또는 테스트 스크립트에서

from models.model import GATreePop
from evolution import Evolution, Selection, Crossover, Mutation
import torch



In [3]:
# --- 1. 시뮬레이션 파라미터 ---
POP_SIZE = 50
MAX_NODES = 256
MAX_DEPTH = 8
MAX_CHILDREN = 3
FEATURE_NUM = {'RSI': (0, 100), 'ATR': (0, 1)}
FEATURE_PAIR = ['SMA_5', 'SMA_20', 'EMA_10']
NUM_GENERATIONS = 100
NUM_ELITES = 5

# --- 2. 집단 초기화 ---
ga_pop = GATreePop(POP_SIZE, MAX_NODES, MAX_DEPTH, MAX_CHILDREN, FEATURE_NUM, FEATURE_PAIR)
ga_pop.make_population()

# --- 3. Evolution 연산자 설정 ---
selection = Selection.TournamentSelection(k=5)
crossover = Crossover.SubtreeCrossover(rate=0.8)
mutation = Mutation.ChainMutation([
    Mutation.NodeParamMutation(prob=0.1),
    Mutation.AddNodeMutation(prob=0.05),
    Mutation.DeleteNodeMutation(prob=0.05)
])

# --- 4. Evolution 엔진 생성 ---
evo_engine = Evolution(
    population=ga_pop,
    selection=selection,
    crossover=crossover,
    mutation=mutation,
    num_elites=NUM_ELITES
)

# --- 5. 진화 루프 (가상) ---
for gen in range(NUM_GENERATIONS):
    print(f"\n===== Generation {gen + 1} =====")
    
    # 실제로는 백테스팅 등을 통해 각 트리의 성능을 평가해야 함
    # 여기서는 임의의 적합도 점수를 생성
    mock_fitness = torch.rand(POP_SIZE)
    
    print(f"Current best fitness: {mock_fitness.max().item():.4f}")
    
    try:
        # 한 세대 진화 실행
        evo_engine.evolve(mock_fitness)
    except NotImplementedError as e:
        print(f"Stopping evolution loop: {e}")
        break

--- Creating Tree 1/50 ---
Tree created with 72 nodes.
--- Creating Tree 2/50 ---
Tree created with 73 nodes.
--- Creating Tree 3/50 ---
Tree created with 150 nodes.
--- Creating Tree 4/50 ---
Tree created with 59 nodes.
--- Creating Tree 5/50 ---
Tree created with 14 nodes.
--- Creating Tree 6/50 ---
Tree created with 52 nodes.
--- Creating Tree 7/50 ---
Tree created with 150 nodes.
--- Creating Tree 8/50 ---
Tree created with 16 nodes.
--- Creating Tree 9/50 ---
Tree created with 121 nodes.
--- Creating Tree 10/50 ---
Tree created with 89 nodes.
--- Creating Tree 11/50 ---
Tree created with 108 nodes.
--- Creating Tree 12/50 ---
Tree created with 103 nodes.
--- Creating Tree 13/50 ---
Tree created with 165 nodes.
--- Creating Tree 14/50 ---
Tree created with 18 nodes.
--- Creating Tree 15/50 ---
Tree created with 157 nodes.
--- Creating Tree 16/50 ---
Tree created with 107 nodes.
--- Creating Tree 17/50 ---
Tree created with 90 nodes.
--- Creating Tree 18/50 ---
Tree created with 131